# Machine Learning Assignment (First Part)
##### Name: Matvey Makhnov<br> 
Task 1: Detection of inconsistencies in flower descriptions in online floristry and delivery platforms is essential for success, customer retention, and satisfaction. Many companies providing online floristry services are increasingly utilizing deep learning solutions to ensure that a flower image displayed on their platform matches the given description or category. <br> <br>To implement a flower classification convolutional neural network (CNN) trained on the Flowers102 dataset

In [4]:
import os 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set a size for our batches 
train_batch = 32
val_batch = 32 
test_batch = 32

train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
]) 


data_path = os.path.join('.', 'Flowers_102_Dataset')

#----------------------------------------------------------------------------------------------------------

#dataset = datasets.Flowers102(root = data_path, transform=None, download=True)
# set a size for our sets (training, validation, test) 
#train_size = int(0.8 * len(dataset))
#val_size = int(0.1 * len(dataset))
#test_size = len(dataset) - (train_size + val_size)
#train_dataset, val_dataset, test_dataset =  random_split(dataset, [train_size,val_size,test_size])

#----------------------------------------------------------------------------------------------------------

# I want to apply to apply different transforms compositions 
# that is why I'll use next code 
train_dataset = datasets.Flowers102(root = data_path, split="train", transform=train_transform, download=True)
val_dataset = datasets.Flowers102(root = data_path, split="val", transform=validation_transform, download=True) 
test_dataset = datasets.Flowers102(root = data_path, split="test", transform=validation_transform, download=True)



#train_dataset, val_dataset, test_dataset =  random_split(dataset, [train_size,val_size,test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=train_batch,shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=val_batch,shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=test_batch, shuffle=False)

print(f"Using device: {device}")

100%|██████████| 345M/345M [00:23<00:00, 14.9MB/s] 


Extracting Flowers_102_Dataset\flowers-102\102flowers.tgz to Flowers_102_Dataset\flowers-102


100%|██████████| 502/502 [00:00<00:00, 252kB/s]


100%|██████████| 15.0k/15.0k [00:00<00:00, 7.53MB/s]


Using device: cpu
